# Intro_to_nlp_spam_url_detection_model

In [ ]:
# Your code here

In [1]:
import logging
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ( train_test_split, GridSearchCV )
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import ( MinMaxScaler, LabelEncoder, StandardScaler, OneHotEncoder )
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import (
        accuracy_score,
        f1_score,
        matthews_corrcoef,
        classification_report,
        ConfusionMatrixDisplay )
from sklearn.inspection import permutation_importance
from sklearn.impute import SimpleImputer
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [4]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from urllib.parse import urlparse
from collections import Counter
from imblearn.over_sampling import SMOTE  # Para lidiar con el desbalance de clases
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/vscode/nltk_data...


### **1. Data loading**
**Objective**: **Obtain the data from source and get a first glimpse of their properties and presentation

In [25]:
#Imprimir la información del dataset
url = ('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')
df_raw = pd.read_csv(url, sep=',')
print("\nInformación del dataset original:")
df_raw.info()


Información del dataset original:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB


In [26]:
# Imprimir las primeras filas y la información del dataset para familiarizarse con los datos.
print("Primeras filas del dataset original:")
print(df_raw.head())

Primeras filas del dataset original:
                                                 url  is_spam
0  https://briefingday.us8.list-manage.com/unsubs...     True
1                             https://www.hvper.com/     True
2                 https://briefingday.com/m/v4n3i4f3     True
3   https://briefingday.com/n/20200618/m#commentform    False
4                        https://briefingday.com/fan     True


### **2. Data preprocessing**
**Objectives**: Perform the data cleaning, data transformation and data reduction steps to avoid data mistmatching, noisy data or data not wrangled

In [27]:
# Paso 2: Preprocesar los enlaces
# -----------------------------
# Se crea una copia del DataFrame para realizar el preprocesamiento sin modificar los datos originales.
df_baking = df_raw.copy()

# 2.1 Función para extraer características de la URL
def extract_url_features(url):
    """
    Extrae varias características de una URL.

    Args:
        url (str): La URL a procesar.

    Returns:
        dict: Un diccionario que contiene las características extraídas.
    """
    parsed_url = urlparse(url)
    features = {
        'longitud_url': len(url),
        'longitud_dominio': len(parsed_url.netloc),
        'cantidad_puntos': url.count('.'),
        'cantidad_guiones': url.count('-'),
        'cantidad_subdominios': parsed_url.netloc.count('.') + 1, # Cuenta el dominio principal como subdominio
        'tiene_http': 1 if 'http' in parsed_url.scheme else 0,
        'tiene_www': 1 if 'www' in parsed_url.netloc else 0,
        'es_ip': 1 if re.match(r'\d+\.\d+\.\d+\.\d+', parsed_url.netloc) else 0,
        'tiene_parametros': 1 if parsed_url.query else 0,
        'tiene_fragmento': 1 if parsed_url.fragment else 0
    }
    return features

In [28]:
# 2.2 Función para tokenizar y limpiar la URL
def tokenize_and_clean_url(url):
    """
    Tokeniza una URL, elimina stopwords y lematiza las palabras.

    Args:
        url (str): La URL a procesar.

    Returns:
        str: La URL procesada como una cadena de texto.
    """
    # Eliminar el esquema (http://, https://)
    url = re.sub(r'^(https?://)?', '', url)
    # Eliminar www.
    url = re.sub(r'www\.', '', url)
    # Tokenizar la URL por delimitadores comunes
    tokens = re.split(r'[./\-?=&_]+', url)
    # Convertir a minúsculas
    tokens = [token.lower() for token in tokens]
    # Eliminar stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and token]  # Añadido 'token' para evitar cadenas vacías
    # Lematizar las palabras
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

In [29]:
def extract_url_features(url):
    features = {}
    try:
        from urllib.parse import urlparse
        parsed_url = urlparse(url)
        features['netloc'] = parsed_url.netloc
        features['path'] = parsed_url.path
        features['query'] = parsed_url.query
        features['fragment'] = parsed_url.fragment
        features['scheme'] = parsed_url.scheme
    except:
        features['netloc'] = None
        features['path'] = None
        features['query'] = None
        features['fragment'] = None
        features['scheme'] = None
    return features

In [30]:
print(nltk.data.path)

['/home/vscode/nltk_data', '/workspaces/Intro_to_nlp_spam_url_detection_model/.venv/nltk_data', '/workspaces/Intro_to_nlp_spam_url_detection_model/.venv/share/nltk_data', '/workspaces/Intro_to_nlp_spam_url_detection_model/.venv/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [31]:
def tokenize_and_clean_url(url):
    try:
        # Convertir a minúsculas
        url = url.lower()
        # Eliminar caracteres especiales (manteniendo letras, números y algunos símbolos de URL)
        url = re.sub(r'[^a-zA-Z0-9./?=&#-]', ' ', url)
        # Tokenizar la URL
        tokens = word_tokenize(url)
        # Obtener las stopwords en inglés (puedes cambiar el idioma si tus URLs son en otro idioma)
        stop_words = set(stopwords.words('english'))
        # Eliminar stopwords y tokens vacíos
        cleaned_tokens = [w for w in tokens if not w in stop_words and len(w) > 0]
        return " ".join(cleaned_tokens)
    except:
        return ""

# Descargar el recurso de stopwords de NLTK (ESTA ES LA LÍNEA AÑADIDA)
try:
    stopwords.words('english') # Intenta acceder a las stopwords para verificar si ya están descargadas
except LookupError:
    nltk.download('stopwords')

### **3. Exploratory Data Analysis** 
**Objective**: Summarize the main characteristics of the dataset using descriptive statistics and data visualization methods

In [32]:
# 2.3 Aplicar las funciones de extracción y limpieza
df_baking['url_features'] = df_baking['url'].apply(extract_url_features)

# Crear columnas separadas para cada característica extraída
df_features = df_baking['url_features'].apply(pd.Series)
df_baking = pd.concat([df_baking, df_features], axis=1)

df_baking['cleaned_url'] = df_baking['url'].apply(tokenize_and_clean_url)

# Imprimir las primeras filas y la información del dataset preprocesado.
print("\nPrimeras filas del dataset preprocesado (df_baking):")
print(df_baking.head())
print("\nInformación del dataset preprocesado (df_baking):")
print(df_baking.info())


Primeras filas del dataset preprocesado (df_baking):
                                                 url  is_spam  \
0  https://briefingday.us8.list-manage.com/unsubs...     True   
1                             https://www.hvper.com/     True   
2                 https://briefingday.com/m/v4n3i4f3     True   
3   https://briefingday.com/n/20200618/m#commentform    False   
4                        https://briefingday.com/fan     True   

                                        url_features  \
0  {'netloc': 'briefingday.us8.list-manage.com', ...   
1  {'netloc': 'www.hvper.com', 'path': '/', 'quer...   
2  {'netloc': 'briefingday.com', 'path': '/m/v4n3...   
3  {'netloc': 'briefingday.com', 'path': '/n/2020...   
4  {'netloc': 'briefingday.com', 'path': '/fan', ...   

                            netloc           path query     fragment scheme  \
0  briefingday.us8.list-manage.com   /unsubscribe                     https   
1                    www.hvper.com              /           

In [33]:
df_baking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           2999 non-null   object
 1   is_spam       2999 non-null   bool  
 2   url_features  2999 non-null   object
 3   netloc        2999 non-null   object
 4   path          2999 non-null   object
 5   query         2999 non-null   object
 6   fragment      2999 non-null   object
 7   scheme        2999 non-null   object
 8   cleaned_url   2999 non-null   object
dtypes: bool(1), object(8)
memory usage: 190.5+ KB


In [24]:
# 2.4 Análisis Bivariado (Ejemplo: Longitud de la URL vs. is_spam)
plt.figure(figsize=(10, 6))
sns.boxplot(x='is_spam', y='longitud_url', data=df_baking)
plt.title('Análisis Bivariado: Longitud de la URL vs. is_spam')
plt.xlabel('¿Es Spam?')
plt.ylabel('Longitud de la URL')
plt.show()

ValueError: Could not interpret value `longitud_url` for `y`. An entry with this name does not appear in `data`.

<Figure size 1000x600 with 0 Axes>